In [ ]:
%run './model/multi_corpus.py'
%run './model/ergm_functions.py'
%run './constants.py'

sns.set(rc = {'figure.figsize':(15,8)})

import multiprocessing as mp
import polars as pl
from itertools import combinations
# mp.set_start_method('forkserver')
import math

import networkx as nx

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import scipy as sp

from itertools import product
from scipy.special import comb
from sklearn.preprocessing import MinMaxScaler

import os

from scipy.spatial.distance import cosine
from scipy.stats import norm

import pytensor.tensor as pt

from IPython.display import display

# RANDOM_SEED = 8927
# rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

In [ ]:
corpora = co_citation_graphs(n_edges=10)
Gs = {field_name: corpus['G'] for (field_name, corpus) in corpora.items()}
Dfs = {field_name: corpus['Df'] for (field_name, corpus) in corpora.items()}

In [ ]:
tables = []
dist = {}

for (field_name, d) in corpora.items():

    print(field_name)

    G = d['G']
    df = d['Df']

    observed = nx.to_numpy_array(G, weight=None)

    βs_xs = {
        'Density': delta_edges(G),
        # 'Triangles': delta_triangles(G),
        # 'Stars': delta_star(G, 3),
        # 'Betweenness': delta_betweenness(G),
        # 'Closeness': delta_closeness(G),
        # 'Eigenvector': delta_eigenvector(G),
        'Centralization': delta_centralization(G),
        # 'Clustering': delta_clustering(G),
        # 'Transitivity': delta_transitivity(G),
        # 'Cliques': delta_cliques(G),
        # 'Components': delta_components(G),
        # 'Gini': delta_gini(G),
        'Louvain': delta_louvain(G),
        # 'Date': date(G, df, 5)
        # 'Geodesic': delta_geodesic(G),
    }

    dist[field_name] = βs_xs

    with pm.Model() as model:

        βs = []
        xs = []
        for β_name, x in βs_xs.items():
            β = pm.Normal(β_name, sigma=1, initval=None)
            βs.append(β)
            xs.append(x)

        μ = sum(β * x for β, x in zip(βs, xs))

        # likelihood = pm.math.sigmoid(μ)
        likelihood = pm.math.exp(μ)

        pm.Bernoulli(name='logit', p=likelihood, observed=observed)

        trace = pm.sample(
            tune=1000,
            draws=2000,
            chains=4,
            # init = 'adapt_diag',
            cores=4,
            # step=pm.Metropolis(),
            step=pm.NUTS(),
            # random_seed=12345,
        )

        trace.to_netcdf(os.path.join(os.path.join(OUTPUT_PATH, f'co_citation_traces'), f'{field_name}.nc'))


# Table

In [ ]:
def p_value_stars(p_value):
    match p_value:
        case _ if p_value <= 0.001:
            stars = '***'
        case _ if p_value <= 0.01:
            stars =  '**'
        case _ if p_value <= 0.05:
            stars =  '*'  
        case _:
            stars = ' '
    return stars

In [ ]:
traces_path = os.path.join(OUTPUT_PATH, f'co_citation_traces')

co_citation_coef = []
co_citation_se = []

for file in os.listdir(traces_path):

    field_name = file.split('.')[0]
    trace = az.from_netcdf(os.path.join(traces_path, file))

    summary = az.summary(trace, kind="stats")
    summary.index.name = 'beta'
    df = pl.from_pandas(summary, include_index=True)
    df = df.with_columns(pl.col('beta').str.replace('_', ' '))

    df = (
        df
        .with_columns((pl.col('mean') / pl.col('sd')).round(4).alias('z_score'))
        .with_columns((pl.col('z_score').abs().apply(norm.sf).round(4).alias('p_value')))
        .with_columns(pl.col('p_value').apply(p_value_stars).alias('significance'))
    )

    df_coef = (
        df.select(
            pl.col('beta'),
            pl.concat_str(
                [
                    pl.col('mean'),
                    pl.col('significance')
                ],
                # sep=''
            ).alias(field_name),
        )
    )

    df_se = (
        df.select(
            pl.col('beta'),
            pl.concat_str(
                [
                    pl.lit('('),
                    pl.col('sd'),
                    pl.lit(')'),
                ]
            ).alias(field_name),
        )
    )

    co_citation_coef.append(df_coef)
    co_citation_se.append(df_se)

iter_dfs = iter(co_citation_coef)
co_citation_df_coef = next(iter_dfs)
for df in iter_dfs:
    co_citation_df_coef = co_citation_df_coef.join(df, on='beta', how='inner')

iter_dfs = iter(co_citation_se)
co_citation_df_se = next(iter_dfs)
for df in iter_dfs:
    co_citation_df_se = co_citation_df_se.join(df, on='beta', how='inner')

columns = sorted(co_citation_df_coef.select(pl.all().exclude('beta')).columns)

In [ ]:
coef_df = co_citation_df_coef.select(pl.concat_str(pl.all(), sep=' & ').alias('all_fields'))
se_df = co_citation_df_se.select(pl.concat_str(pl.all().exclude('beta').alias('all_fields'), sep=' & '))

coef_df = coef_df['all_fields'].to_list()
se_df = se_df['all_fields'].to_list()

row_str = ''
for i in range(len(se_df)):
    row_str += f'{coef_df[i]} \\\ \n & {se_df[i]} \\\ \n \\addlinespace[0.5em] \n'

alignments = ''.join(['c']*len(columns)*2)

new_columns = []
for col in columns:
    if ' & ' in col:
        first, second = col.split(' & ')
        s = f'\\begin{{tabular}}{{cc}} {first} \& \\\ {second} \\end{{tabular}}'
        new_columns.append(s)
    elif ' ' in col:
        first, second = col.split(' ')
        s = f'\\begin{{tabular}}{{cc}} {first} \\\ {second} \\end{{tabular}}'
        new_columns.append(s)
    else:
        s = f'\\begin{{tabular}}{{cc}} {col} \\end{{tabular}}'
        new_columns.append(s)
new_columns = ' \n& '.join(new_columns)


table_str = f"""
\\begin{{tabular}}{{l*{{{len(columns)*2}}}{{c}}}}
\\toprule
\\addlinespace[0.7em]
& {new_columns} \\\ 
\\midrule
\\midrule
\\addlinespace[0.5em]
{row_str}
\\bottomrule
\end{{tabular}}
"""

with open(os.path.join(LATEX_TABLE_PATH, 'co_citation_ergm_model.tex'), "w+") as file:
    file.write(table_str)

In [ ]:
print(table_str)

In [ ]:
pm.model_to_graphviz(model)

In [ ]:
dfs = []

for field_name, features in dist.items():
    df = pl.DataFrame()
    for feature_name, mat in features.items():
        df = df.with_columns(
            pl.Series(
                feature_name,
                mat.reshape(np.multiply(*mat.shape))
            )
        )
    df = df.with_columns(
        pl.Series(
            'Field',
            np.full((len(df)), field_name)
        )
    )
    dfs.append(df)
df = pl.concat(dfs)

# Univariate

In [ ]:
cols = df.select(pl.all().exclude('Field')).columns
n_cols = len(cols)

fig, axs = plt.subplots(n_cols, figsize=(16, 10*n_cols))
for col, ax in zip(cols, axs):
    sns.stripplot(data=df.to_pandas(), x=col, y="Field", ax=ax)
    ax.spines['top'].set_color('k')
    ax.spines['top'].set_linewidth(1)
    ax.spines['bottom'].set_color('k')
    ax.spines['bottom'].set_linewidth(1)
    ax.spines['right'].set_color('k')
    ax.spines['right'].set_linewidth(1)
    ax.spines['left'].set_color('k')
    ax.spines['left'].set_linewidth(1)

plt.plot()

# Bivariate

In [ ]:
df.to_pandas().corr()

In [ ]:
# sns.set(rc={'figure.figsize':(16, 12)})
sns.set_style('darkgrid', {'axes.linewidth': 1, 'axes.edgecolor': 'black', 'axes.spines.left': True, 'axes.spines.bottom': True, 'axes.spines.right': True, 'axes.spines.top': True})

pp = sns.pairplot(df.to_pandas(), vars=None, hue='Field', corner=True, grid_kws={'layout_pad': 0.5}) # , corner=True , grid_kws={"despine": False}

pp.map_lower(plt.scatter, alpha = 0.6, linewidth=1, edgecolor='k')

# pp.map_lower(sns.scatterplot)
# g.map_lower(sns.kdeplot, hue=None, levels=5, color=".2")
# pp.map_diag(plt.kdeplot, alpha=1)
# pp.map_upper(sns.kdeplot, shade=True)

# for ax in pp.axes.flat:
#     if ax:
#         ax.spines['top'].set_color('k')
#         ax.spines['top'].set_linewidth(1)
#         ax.spines['bottom'].set_color('k')
#         ax.spines['bottom'].set_linewidth(1)
#         ax.spines['right'].set_color('k')
#         ax.spines['right'].set_linewidth(1)
#         ax.spines['left'].set_color('k')
#         ax.spines['left'].set_linewidth(1)

plt.show()

In [ ]:
# sns.set_style('darkgrid', {'axes.linewidth': 1, 'axes.edgecolor':'black'})

# fig, ax = plt.subplots(1)
jp = sns.jointplot(data=df, x="Eigenvector_centrality", y="Transitivity", hue="Field", height=15, space=0)
jp.plot_marginals(sns.kdeplot)

jp.ax_marg_x.set_facecolor('w')
jp.ax_marg_y.set_facecolor('w')

# for col, ax in zip(cols, axs):
#     sns.stripplot(data=df.to_pandas(), x=col, y="Field", ax=ax)
jp.ax.spines['top'].set_color('k')
jp.spines['top'].set_linewidth(1)
jp.spines['bottom'].set_color('k')
jp.spines['bottom'].set_linewidth(1)
jp.spines['right'].set_color('k')
jp.spines['right'].set_linewidth(1)
jp.spines['left'].set_color('k')
jp.spines['left'].set_linewidth(1)

plt.show()

# Embedding 

In [ ]:
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler

In [ ]:
traces_path = os.path.join(OUTPUT_PATH, f'co_citation_traces')

co_citation_coef = []

for file in os.listdir(traces_path):

    field_name = file.split('.')[0]
    trace = az.from_netcdf(os.path.join(traces_path, file))

    summary = az.summary(trace, kind="stats")
    summary.index.name = 'beta'
    df = pl.from_pandas(summary, include_index=True)
    df = df.with_columns(pl.col('beta').str.replace('_', ' '))

    df = (
        df
        .with_columns((pl.col('mean') / pl.col('sd')).round(4).alias('z_score'))
        .with_columns((pl.col('z_score').abs().apply(norm.sf).round(4).alias('p_value')))
    )

    df_coef = (
        df.select(
            pl.col('beta'),
            pl.when(pl.col('p_value') > 0.05).then(0).otherwise(pl.col('mean')).alias(field_name)
        )
    )

    co_citation_coef.append(df_coef)

iter_dfs = iter(co_citation_coef)
co_citation_df_coef = next(iter_dfs)
for df in iter_dfs:
    co_citation_df_coef = co_citation_df_coef.join(df, on='beta', how='inner')

columns = co_citation_df_coef['beta'].to_list()
vec_df = co_citation_df_coef.select(pl.all().exclude('beta')).transpose(include_header=True, header_name='Field', column_names=columns)
field_names = vec_df['Field'].to_list()
vecs = vec_df.select(pl.all().exclude('Field')).to_numpy()

scaled_data = StandardScaler().fit_transform(vecs)

In [ ]:
reducer = umap.UMAP(random_state=42)
embs = reducer.fit_transform(scaled_data)

sns.set()

fig, ax = plt.subplots(1, 1, figsize=(12, 8))

plt.scatter(embs[:, 0], embs[:, 1], alpha=1, edgecolor='k')

# ax.set_title(f'{field_name.capitalize()}', fontweight='semibold', fontsize=20)

for field_name, (x, y) in zip(field_names, embs):
    plt.text(x, y+0.05, field_name)


ax.spines['top'].set_color('k')
ax.spines['top'].set_linewidth(1)
ax.spines['bottom'].set_color('k')
ax.spines['bottom'].set_linewidth(1)
ax.spines['right'].set_color('k')
ax.spines['right'].set_linewidth(1)
ax.spines['left'].set_color('k')
ax.spines['left'].set_linewidth(1)

# fig.tight_layout()
# plt.savefig(f'{OBSIDIAN_IMG_PATH}/co_citation_umap_node2vec.png')
plt.plot()